## **Car Speed Detection and Tracking**

**Elahe Khodaverdi - Fereshte Bagheri**

---

## **Description**

The project uses Haar Cascade for detecting cars and Dlib’s correlation tracker for continuous tracking across frames. Every 10 frames, the system detects new cars, and their positions are tracked. The speed of each car is calculated based on the distance it travels between consecutive frames. This speed is displayed as text on the video feed. 
---

## **Requirements**
- **Libraries**: OpenCV, Dlib, NumPy
- **File Structure**:
  - Model files (e.g., `myhaar.xml`) are located in the `models/` directory.
  - Video files (e.g., `cars.mp4`) are located in the `videos/` directory.
  - Output video is saved in the current directory as `outpy.avi`.

---

## **Outputs**
1. **Video**:
   - Original video with detected cars tracked and their speeds displayed.

### **Sample Outputs**
- Processed video saved as `outpy.avi` with bounding boxes and speed annotations.

---

## **Algorithm Details**

1. **Car Detection**:
   - A Haar Cascade model is used to detect cars in the video frames. Detection occurs every 10 frames to identify new cars.

2. **Tracking with Dlib**:
   - Dlib’s `correlation_tracker` is used to continuously track the detected cars. A tracker is created for each detected car, and its position is updated in subsequent frames.

3. **Speed Estimation**:
   - The speed is estimated by calculating the distance between a car’s position in two consecutive frames and converting the distance from pixels to meters. The speed is displayed on the video feed in km/hr.

4. **Frame Processing**:
   - The system tracks multiple cars and removes trackers for cars no longer detected. It updates the positions of the remaining tracked cars and recalculates their speeds as they move across the frame.

5. **Output Generation**:
   - The processed video is saved and displayed, with bounding boxes around tracked cars and speed annotations displayed on the screen.

# Vehicle Speed Check
### Complete # TODO

# Requirements

In [3]:
import cv2
import dlib
import time
import math
import os

# Loading Model

In [4]:
# Define the path to the Haar Cascade model file using os.path.join for cross-platform compatibility
MODEL_ADDRESS = os.path.join("models", "myhaar.xml")

# Define the path to the video file using os.path.join for cross-platform compatibility
VIDEO_ADDRESS = os.path.join("videos", "cars.mp4")

# Load the Haar Cascade model for car detection
carCascade = cv2.CascadeClassifier(MODEL_ADDRESS) # TODO

# Initialize video capture with the specified video file path
video = cv2.VideoCapture(VIDEO_ADDRESS) # TODO

# Constants for video frame dimensions
WIDTH = 1280  # Width of the video frame
HEIGHT = 720  # Height of the video frame

# Speed Detection

In [5]:
# Function to estimate the speed of an object based on two locations
def estimateSpeed(location1, location2):
    d_pixels = math.sqrt(
        math.pow(location2[0] - location1[0], 2)
        + math.pow(location2[1] - location1[1], 2)
    )
    ppm = 8.8  # Pixels per meter
    d_meters = d_pixels / ppm  # Distance in meters
    fps = 18  # Frames per second
    speed = d_meters * fps * 3.6  # Speed in km/hr
    return speed

# Function to track multiple objects in a video
def trackMultipleObjects():
    rectangleColor = (0, 255, 0)  # Color of the rectangle around tracked objects
    frameCounter = 0  # Counter for frames
    currentCarID = 0  # ID for the current car being tracked
    fps = 0  # Frames per second

    carTracker = {}  # Dictionary to store car trackers
    carLocation1 = {}  # Dictionary to store initial car locations
    carLocation2 = {}  # Dictionary to store current car locations
    speed = [None] * 1000  # List to store speeds of cars

    out = cv2.VideoWriter(
        "outpy.avi", cv2.VideoWriter_fourcc("M", "J", "P", "G"), 10, (WIDTH, HEIGHT)
    )  # Video writer to save output video

    while True:
        start_time = time.time()  # Record the start time for FPS calculation
        rc, image = video.read()  # Read a frame from the video
        if type(image) == type(None):
            break

        image = cv2.resize(image, (WIDTH, HEIGHT))  # Resize the frame to desired dimensions
        resultImage = image.copy()  # Make a copy of the frame for result

        frameCounter = frameCounter + 1  # Increment frame counter

        carIDtoDelete = []  # List to store IDs of cars to delete

        # Update the trackers and check their quality
        for carID in carTracker.keys():
            trackingQuality = carTracker[carID].update(image)
            if trackingQuality < 7:
                carIDtoDelete.append(carID)

        # Remove cars that are no longer tracked
        for carID in carIDtoDelete:
            print("Removing carID " + str(carID) + " from list of trackers.")
            print("Removing carID " + str(carID) + " previous location.")
            print("Removing carID " + str(carID) + " current location.")
            carTracker.pop(carID, None)
            carLocation1.pop(carID, None)
            carLocation2.pop(carID, None)

        # Detect new cars every 10 frames
        if not (frameCounter % 10):
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            cars = carCascade.detectMultiScale(gray, 1.1, 13, 18, (24, 24))
            for _x, _y, _w, _h in cars:
                x = int(_x)
                y = int(_y)
                w = int(_w)
                h = int(_h)
                x_bar = x + 0.5 * w
                y_bar = y + 0.5 * h
                matchCarID = None

                # Check if the detected car matches an existing car
                for carID in carTracker.keys():
                    trackedPosition = carTracker[carID].get_position()
                    t_x = int(trackedPosition.left())
                    t_y = int(trackedPosition.top())
                    t_w = int(trackedPosition.width())
                    t_h = int(trackedPosition.height())
                    t_x_bar = t_x + 0.5 * t_w
                    t_y_bar = t_y + 0.5 * t_h
                    if (
                        (t_x <= x_bar <= (t_x + t_w))
                        and (t_y <= y_bar <= (t_y + t_h))
                        and (x <= t_x_bar <= (x + w))
                        and (y <= t_y_bar <= (y + h))
                    ):
                        matchCarID = carID

                # If no match is found, create a new tracker
                if matchCarID is None:
                    print("Creating new tracker " + str(currentCarID))
                    tracker = dlib.correlation_tracker()
                    tracker.start_track(image, dlib.rectangle(x, y, x + w, y + h))
                    carTracker[currentCarID] = tracker
                    carLocation1[currentCarID] = [x, y, w, h]
                    currentCarID = currentCarID + 1

        # Update the positions of tracked cars
        for carID in carTracker.keys():
            trackedPosition = carTracker[carID].get_position()
            t_x = int(trackedPosition.left())
            t_y = int(trackedPosition.top())
            t_w = int(trackedPosition.width())
            t_h = int(trackedPosition.height())
            cv2.rectangle(
                resultImage, (t_x, t_y), (t_x + t_w, t_y + t_h), rectangleColor, 4
            )
            carLocation2[carID] = [t_x, t_y, t_w, t_h]

        end_time = time.time()  # Record the end time for FPS calculation
        if not (end_time == start_time):
            fps = 1.0 / (end_time - start_time)  # Calculate FPS

        # Calculate the speed of the cars
        for i in carLocation1.keys():
            if frameCounter % 1 == 0:
                [x1, y1, w1, h1] = carLocation1[i]
                [x2, y2, w2, h2] = carLocation2[i]
                carLocation1[i] = [x2, y2, w2, h2]
                if [x1, y1, w1, h1] != [x2, y2, w2, h2]:
                    if (speed[i] == None or speed[i] == 0) and y1 >= 275 and y1 <= 285:
                        speed[i] = estimateSpeed([x1, y1, w1, h1], [x2, y2, w2, h2])
                    if speed[i] != None and y1 >= 180:
                        cv2.putText(
                            resultImage,
                            str(int(speed[i])) + " km/hr",
                            (int(x1 + w1 / 2), int(y1 - 5)),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.75,
                            (255, 255, 255),
                            2,
                        )

        # Display the result
        cv2.imshow("result", resultImage)
        if cv2.waitKey(33) == 27:
            break

    cv2.destroyAllWindows()

# Main function to start tracking
if __name__ == "__main__":
    trackMultipleObjects()


Creating new tracker 0
Creating new tracker 1
Creating new tracker 2
Creating new tracker 3
Creating new tracker 4
Creating new tracker 5
Creating new tracker 6
Creating new tracker 7
Creating new tracker 8
Creating new tracker 9
Creating new tracker 10
Creating new tracker 11
Removing carID 11 from list of trackers.
Removing carID 11 previous location.
Removing carID 11 current location.
Creating new tracker 12
Removing carID 2 from list of trackers.
Removing carID 2 previous location.
Removing carID 2 current location.
Removing carID 1 from list of trackers.
Removing carID 1 previous location.
Removing carID 1 current location.
Creating new tracker 13
Removing carID 4 from list of trackers.
Removing carID 4 previous location.
Removing carID 4 current location.
Creating new tracker 14
Creating new tracker 15
Creating new tracker 16
Creating new tracker 17
Removing carID 9 from list of trackers.
Removing carID 9 previous location.
Removing carID 9 current location.
Creating new tracker